In [10]:
# ========================= SNAKE AI EXPERIMENT SUITE ========================= #
# BFS | DFS | A* | Hamiltonian — Full Benchmark in Google Colab (NO GUI)
# Implements your Wrapped Manhattan Distance Heuristic for A*
# ============================================================================ #

import random, time
from heapq import heappush, heappop
from collections import deque


# ========================== GRID + SIMULATION SETTINGS ======================= #
GRID_SIZE = 35                 # bigger grid → better difference
RUNS = 12                      # repeat test for fair average
GROWTH_RATE = 2                # snake grows faster → tests intelligence
MAX_STEPS = 2500               # prevents infinite looping

# ============================================================================ #
# FOOD + NEIGHBOURS  (Wrap-around World)
# ============================================================================ #
def generate_food(snake):
    while True:
        f = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
        if f not in snake: return f

def get_neighbors(pos, snake):
    x,y = pos
    ns=[]
    for dx,dy in [(1,0),(-1,0),(0,1),(0,-1)]:
        nx,ny = (x+dx)%GRID_SIZE, (y+dy)%GRID_SIZE   # WRAPPING
        if (nx,ny) not in snake: ns.append((nx,ny))
    return ns


# ============================================================================ #
# DEPTH FIRST SEARCH  (Fast but reckless)
# ============================================================================ #
def dfs(snake, food):
    start=snake[0]
    st=[start]; parent={start:None}
    while st:
        cur=st.pop()
        if cur==food:
            path=[]
            while cur: path.append(cur); cur=parent[cur]
            return path[::-1][1:]
        for nxt in get_neighbors(cur,snake):
            if nxt not in parent:
                parent[nxt]=cur; st.append(nxt)
    return []


# ============================================================================ #
# BREADTH FIRST SEARCH (Shortest path but blind & slow)
# ============================================================================ #
def bfs(snake, food):
    start=snake[0]
    q=deque([start]); parent={start:None}
    while q:
        cur=q.popleft()
        if cur==food:
            path=[]
            while cur: path.append(cur); cur=parent[cur]
            return path[::-1][1:]
        for nxt in get_neighbors(cur,snake):
            if nxt not in parent:
                parent[nxt]=cur; q.append(nxt)
    return []


# ============================================================================ #
# ★ ★ A* WITH YOUR UPDATED HEURISTIC ★ ★
# Wrapped Manhattan Distance (REGULAR + WRAP AROUND)
# ============================================================================ #
def heuristic(a, b):
    dx = min(abs(a[0]-b[0]), GRID_SIZE-abs(a[0]-b[0]))   # horizontal wrap distance
    dy = min(abs(a[1]-b[1]), GRID_SIZE-abs(a[1]-b[1]))   # vertical wrap distance
    return dx + dy

def astar(snake, food):
    start=snake[0]
    pq=[(0,start)]
    parent={start:None}
    g={start:0}

    while pq:
        f,cur = heappop(pq)
        if cur == food:
            path=[]
            while cur: path.append(cur); cur=parent[cur]
            return path[::-1][1:]

        for nxt in get_neighbors(cur,snake):
            ng = g[cur] + 1
            if nxt not in g or ng < g[nxt]:
                g[nxt]=ng; parent[nxt]=cur
                heappush(pq,(ng + heuristic(nxt,food), nxt))
    return []


# ============================================================================ #
# HAMILTONIAN CYCLE (Never dies, but inefficient & long path)
# ============================================================================ #
def generate_hamiltonian_cycle(n):
    cyc=[]
    for y in range(n):
        if y%2==0:       # alternate horizontal rows
            for x in range(n): cyc.append((x,y))
        else:
            for x in reversed(range(n)): cyc.append((x,y))
    return cyc

HAM = generate_hamiltonian_cycle(GRID_SIZE)

def hamiltonian(snake,food):
    head = snake[0]
    i = HAM.index(head)
    return [HAM[(i+1)%len(HAM)]]  # follow cycle


# ============================================================================ #
# EVALUATION FUNCTION – Run All Algorithms
# ============================================================================ #
def evaluate(name,algo):
    total_food=total_time=survival=0

    for _ in range(RUNS):
        snake=[(GRID_SIZE//2,GRID_SIZE//2)]
        food=generate_food(snake)
        steps=0
        start=time.time()

        while steps<MAX_STEPS:
            path = algo(snake,food)
            if not path: break

            for step in path:
                snake.insert(0,step)
                snake.pop()
                steps+=1

                if step==food:
                    for _ in range(GROWTH_RATE):
                        snake.append(snake[-1])
                    food=generate_food(snake)

                if steps>=MAX_STEPS: break

        end=time.time()
        total_food+=len(snake)
        survival+=steps
        total_time+=(end-start)

    print(f"\n{name} Results")
    print("Avg Food:", round(total_food/RUNS,2))
    print("Avg Survival Steps:", round(survival/RUNS,2))
    print("Avg Time:", round(total_time/RUNS,4),"sec")


# ============================================================================ #
# RUN ALL FOUR ALGORITHMS
# ============================================================================ #
evaluate("DFS", dfs)
evaluate("BFS", bfs)
evaluate("A* with Wrapped Manhattan Heuristic", astar)
evaluate("Hamiltonian Cycle", hamiltonian)




DFS Results
Avg Food: 17.83
Avg Survival Steps: 2500.0
Avg Time: 0.0163 sec

BFS Results
Avg Food: 135.17
Avg Survival Steps: 1315.0
Avg Time: 0.3883 sec

A* with Wrapped Manhattan Heuristic Results
Avg Food: 127.17
Avg Survival Steps: 1203.83
Avg Time: 0.0601 sec

Hamiltonian Cycle Results
Avg Food: 7.83
Avg Survival Steps: 2500.0
Avg Time: 0.0396 sec
